In [7]:
# Additional imports for loading PDF documents and QA chain.
from dotenv import load_dotenv
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter



from transformers import pipeline

import gradio as gr

In [8]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"
# Store the API key in a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [9]:
# Specify the folder path
folder_path = "embedded_resources/"

# Get all files in the folder and add them to a list
file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Print the list of files
print(len(file_list))

71


In [13]:
def ada_qa_chatbot(query):
    all_documents = []

    for file in file_list:
        full_path = os.path.join(folder_path, file)
        if os.path.exists(full_path):
            pdf_loader = PyPDFLoader(full_path)
            documents = pdf_loader.load()
            all_documents.extend(documents)
        else:
            print(f"Warning: file not found - {full_path}")

    llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.0)

    chain = load_qa_chain(llm)

    result = chain.invoke({"input_documents": all_documents, "question": query})
    return result["output_text"]

In [11]:
# Create an instance of the Gradio Interface application function with parameters. 
app = gr.Interface(fn=ada_qa_chatbot, 
                   title="American Disabilities Act Chatbot",
                   description="Ask the chatbot about the ADA requirements per '2021 Accessibility Pocketbook 2021 IBC®, 2021 IEBC® and ICC A117.1-2017'.",
                   inputs=["text"], 
                   outputs=gr.Textbox(lines=20, label="Summarized Response", show_copy_button=True))
# Launch the app
app.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\elcoo\anaconda3\Lib\site-packages\gradio\analytics.py:102: UserWarning: IMPORTANT: You are using gradio version 5.0.0, however version 5.0.1 is available, please upgrade. 
--------
  warnings.warn(


C:\Users\elcoo\AppData\Local\Temp\ipykernel_31572\1695826883.py:15: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm)


In [5]:
def ada_conversation_chatbot(query, chat_history=[]):
    # Initialize the document store if it doesn't exist
    if not hasattr(ada_conversation_chatbot, 'vectorstore'):
        all_documents = []
        for file in file_list:
            full_path = os.path.join(folder_path, file)
            if os.path.exists(full_path):
                pdf_loader = PyPDFLoader(full_path)
                documents = pdf_loader.load()
                all_documents.extend(documents)
            else:
                print(f"Warning: file not found - {full_path}")

        # Split documents into chunks
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        texts = text_splitter.split_documents(all_documents)

        # Create embeddings and vector store
        embeddings = HuggingFaceEmbeddings()
        ada_conversation_chatbot.vectorstore = FAISS.from_documents(texts, embeddings)

    # Initialize or retrieve memory
    if not hasattr(ada_conversation_chatbot, 'memory'):
        ada_conversation_chatbot.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # Initialize LLM
    llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.0)

    # Create ConversationalRetrievalChain
    chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=ada_conversation_chatbot.vectorstore.as_retriever(),
        memory=ada_conversation_chatbot.memory
    )

    # Get the response
    result = chain({"question": query})
    
    # Update chat history
    chat_history.append((query, result['answer']))

    return result['answer'], chat_history

In [ ]:
# Modify the Gradio interface
app = gr.Interface(
    fn=ada_conversation_chatbot,
    inputs=[
        gr.Textbox(lines=2, label="Your Question"),
        gr.State([])  # For storing chat history
    ],
    outputs=[
        gr.Textbox(lines=20, label="Response", show_copy_button=True),
        gr.State()  # For updating chat history
    ],
    title="American Disabilities Act Chatbot",
    description="Ask the chatbot about the ADA requirements per '2021 Accessibility Pocketbook 2021 IBC®, 2021 IEBC® and ICC A117.1-2017'.",
)

# Launch the app
app.launch()